<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Import-dataset" data-toc-modified-id="Import-dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Import dataset</a></span></li><li><span><a href="#Sentence-embedding" data-toc-modified-id="Sentence-embedding-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Sentence embedding</a></span><ul class="toc-item"><li><span><a href="#Infersent" data-toc-modified-id="Infersent-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Infersent</a></span></li></ul></li><li><span><a href="#Dimensionality-reduction" data-toc-modified-id="Dimensionality-reduction-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Dimensionality reduction</a></span><ul class="toc-item"><li><span><a href="#TSNE" data-toc-modified-id="TSNE-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>TSNE</a></span></li></ul></li><li><span><a href="#Blocking" data-toc-modified-id="Blocking-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Blocking</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Evaluation</a></span><ul class="toc-item"><li><span><a href="#bilstm,-rnn_dim-2048" data-toc-modified-id="bilstm,-rnn_dim-2048-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>bilstm, rnn_dim 2048</a></span></li><li><span><a href="#bilstrm,-rnn_dim:-1024" data-toc-modified-id="bilstrm,-rnn_dim:-1024-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>bilstrm, rnn_dim: 1024</a></span></li><li><span><a href="#bilstm,-rnn_dim:-300" data-toc-modified-id="bilstm,-rnn_dim:-300-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>bilstm, rnn_dim: 300</a></span></li><li><span><a href="#lstm,-rnn_dim:-2048" data-toc-modified-id="lstm,-rnn_dim:-2048-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>lstm, rnn_dim: 2048</a></span></li><li><span><a href="#lstm,-rnn_dim:-1024" data-toc-modified-id="lstm,-rnn_dim:-1024-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>lstm, rnn_dim: 1024</a></span></li><li><span><a href="#lstm,-rnn_dim:-300" data-toc-modified-id="lstm,-rnn_dim:-300-6.6"><span class="toc-item-num">6.6&nbsp;&nbsp;</span>lstm, rnn_dim: 300</a></span></li><li><span><a href="#Searching" data-toc-modified-id="Searching-6.7"><span class="toc-item-num">6.7&nbsp;&nbsp;</span>Searching</a></span></li></ul></li><li><span><a href="#Best" data-toc-modified-id="Best-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Best</a></span></li></ul></div>

# Setup 

In [1]:
import os
import sys
import time
import numpy as np
import matplotlib.pyplot as plt

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from preprocessing_datasets.preprocessing_utilities import get_labels_by

params = {}

# Import dataset

In [2]:
from preprocessing_datasets.preprocessing_restaurant import clean_restaurant
table, pairs = clean_restaurant()

In [3]:
table

,citruname,addr,city,phone,type
0,arnie morton's of chicago,"""435 s. la cienega blv.""","""los angeles""","""310/246-1501""","""american"""
1,arnie morton's of chicago,"""435 s. la cienega blvd.""","""los angeles""","""310-246-1501""","""steakhouses"""
2,art's delicatessen,"""12224 ventura blvd.""","""studio city""","""818/762-1221""","""american"""
3,art's deli,"""12224 ventura blvd.""","""studio city""","""818-762-1221""","""delis"""
4,hotel bel-air,"""701 stone canyon rd.""","""bel air""","""310/472-1211""","""californian"""
...,...,...,...,...,...
859,ti couz,"""3108 16th st.""","""san francisco""","""415-252-7373""","""french"""
860,trio cafe,"""1870 fillmore st.""","""san francisco""","""415-563-2248""","""american"""
861,tu lan,"""8 sixth st.""","""san francisco""","""415-626-0927""","""vietnamese"""
862,vicolo pizzeria,"""201 ivy st.""","""san francisco""","""415-863-2382""","""pizza"""


# Sentence embedding

## Infersent

Setup model

In [4]:
from embedding_algorithms.inferSent import set_RNN_embedding
start = time.time()
model_type = "bilstm" 
char_level = False
model_version = 2
rnn_dim = 1024
verbose = 1
set_RNN_embedding(model_type, char_level, model_version, rnn_dim, verbose)
print("TIME: {0}".format(time.time() - start))

params['embedding'] = {
    'name': 'Infersent',
    'model_type': model_type,
    'char_level': char_level,
    'model_version': model_version,
    'rnn_dim': rnn_dim,
    'verbose': verbose
}

Vocab size : 2196017
TIME: 166.85381603240967


Embedding

In [29]:
from embedding_algorithms.inferSent import RNN_embedding
attr_list = ['citruname', 'city', 'type']
params['attr_list'] = attr_list
embeddings_tokens = RNN_embedding(table, attr_list, model_type, char_level)
embeddings_tokens = np.array(embeddings_tokens)

attrs: ['citruname', 'city', 'type']


# Dimensionality reduction

## TSNE

In [30]:
from dimensionality_reduction_algorithms.tsne import tsne_dim_reduction
tsne_embeddings = tsne_dim_reduction(
    embeddings_tokens, 
    num_components=2,
    early_exaggeration=12,
    verbose=1,
    perplexity=10,
    method="barnes_hut")

params['reduction'] = {
    'name': 'TSNE',
    'num_components': 2,
    'early_exaggeration': 12,
    'verbose': 1,
    'perplexity': 10,
    'method': "barnes_hut"
}

starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 10
early_exaggeration: 12


# Blocking

hierarchy_cluster

In [31]:
from cluster_algorithms.hierarchy_cluster import hierarchy_cluster
num_clusters = 56
blocks = hierarchy_cluster(tsne_embeddings, {'num_clusters': num_clusters})

params['blocking'] = {
    'name': 'hierarchy',
    'num_clusters': num_clusters
}

clustering with NUM_CLUSTERS = 56, 


# Evaluation

## bilstm, rnn_dim 2048

In [39]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9894103257370929
(PC) Pair completeness is: 0.7857142857142857
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8758752172662914
(PQ) Pair quality - Precision is: 0.022289766970618033
(FM) Fmeasure is: 0.04334975369458128

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 100}


In [54]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.989367409124072
(PC) Pair completeness is: 0.7321428571428571
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8415381492332136
(PQ) Pair quality - Precision is: 0.020686175580221997
(FM) Fmeasure is: 0.04023552502453385

['citruname', 'addr', 'city']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 100}


## bilstrm, rnn_dim: 1024

In [12]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9922803742328655
(PC) Pair completeness is: 0.7857142857142857
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8769979832131264
(PQ) Pair quality - Precision is: 0.030576789437109102
(FM) Fmeasure is: 0.0588628762541806

['citruname', 'addr', 'city']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


In [16]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9923420668640831
(PC) Pair completeness is: 0.875
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9299841993751572
(PQ) Pair quality - Precision is: 0.034325744308231175
(FM) Fmeasure is: 0.06605999325918437

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


In [8]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.992468134414832
(PC) Pair completeness is: 0.8392857142857143
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9094719005930528
(PQ) Pair quality - Precision is: 0.03347578347578348
(FM) Fmeasure is: 0.06438356164383562

['citruname']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


In [30]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9917975623363804
(PC) Pair completeness is: 0.4642857142857143
(RM) Reference metric (Harmonic mean RR and PC) is: 0.6324877801280961
(PQ) Pair quality - Precision is: 0.017004578155657292
(FM) Fmeasure is: 0.03280757097791798

['addr', 'city']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


In [34]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9924466761083215
(PC) Pair completeness is: 0.6964285714285714
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8184952936804223
(PQ) Pair quality - Precision is: 0.027698863636363636
(FM) Fmeasure is: 0.05327868852459017

['addr']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


In [19]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9803468735247415
(PC) Pair completeness is: 0.8928571428571429
(RM) Reference metric (Harmonic mean RR and PC) is: 0.934558863689503
(PQ) Pair quality - Precision is: 0.013648150675583458
(FM) Fmeasure is: 0.026885334050275576

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 10, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 56}


In [24]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9806258315093773
(PC) Pair completeness is: 0.8214285714285714
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8939952917841398
(PQ) Pair quality - Precision is: 0.012737089851862107
(FM) Fmeasure is: 0.025085207907293797

['citruname', 'addr', 'city']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 10, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 56}


In [28]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9790191408094073
(PC) Pair completeness is: 0.9464285714285714
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9624480383946711
(PQ) Pair quality - Precision is: 0.013551521350038353
(FM) Fmeasure is: 0.026720443660196623

['citruname']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 10, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 56}


In [32]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9782600532166001
(PC) Pair completeness is: 0.8660714285714286
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9187535865106893
(PQ) Pair quality - Precision is: 0.011967921036397286
(FM) Fmeasure is: 0.023609589874650113

['citruname', 'city', 'type']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 10, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 56}


## bilstm, rnn_dim: 300

In [9]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9920550620145058
(PC) Pair completeness is: 0.8303571428571429
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9040325834619262
(PQ) Pair quality - Precision is: 0.03139770425388251
(FM) Fmeasure is: 0.060507482108002594

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 300, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


In [15]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9924654521265182
(PC) Pair completeness is: 0.8482142857142857
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9146875007806573
(PQ) Pair quality - Precision is: 0.03381986472054112
(FM) Fmeasure is: 0.06504621704895584

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 300, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


## lstm, rnn_dim: 2048

In [33]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9924064417836145
(PC) Pair completeness is: 0.7589285714285714
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8601045459120267
(PQ) Pair quality - Precision is: 0.03002472624514306
(FM) Fmeasure is: 0.057764186204553175

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'lstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 2048, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


## lstm, rnn_dim: 1024

In [26]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.992301832539376
(PC) Pair completeness is: 0.7410714285714286
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8484803049779988
(PQ) Pair quality - Precision is: 0.0289198606271777
(FM) Fmeasure is: 0.05566733735747821

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'lstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


## lstm, rnn_dim: 300

In [20]:
from evaluation import calc_index
calc_index(blocks,table,pairs)
print()
print(params['attr_list'])
print(params['embedding'])
print(params['reduction'])
print(params['blocking'])

(RR) Reduction ratio is: 0.9925298270460495
(PC) Pair completeness is: 0.7232142857142857
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8367352037856011
(PQ) Pair quality - Precision is: 0.029084380610412925
(FM) Fmeasure is: 0.05591991715567828

['citruname', 'addr']
{'name': 'Infersent', 'model_type': 'lstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 300, 'verbose': 1}
{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
{'name': 'hierarchy', 'num_clusters': 136}


## Searching

In [33]:
for i in range(1,300,5):
    num_clusters = int(i)
    blocks = hierarchy_cluster(tsne_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

clustering with NUM_CLUSTERS = 1, 
(RR) Reduction ratio is: 0.0
(PC) Pair completeness is: 1.0
(RM) Reference metric (Harmonic mean RR and PC) is: 0
(PQ) Pair quality - Precision is: 0.0003004162911463027
(FM) Fmeasure is: 0.0006006521366054572
clustering with NUM_CLUSTERS = 6, 
(RR) Reduction ratio is: 0.8283415947813398
(PC) Pair completeness is: 0.875
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8510317574047334
(PQ) Pair quality - Precision is: 0.0015313217807084707
(FM) Fmeasure is: 0.0030572930477780033
clustering with NUM_CLUSTERS = 11, 
(RR) Reduction ratio is: 0.8987221578473027
(PC) Pair completeness is: 0.875
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8867024461946068
(PQ) Pair quality - Precision is: 0.0025954764553207266
(FM) Fmeasure is: 0.005175600739371534
clustering with NUM_CLUSTERS = 16, 
(RR) Reduction ratio is: 0.9277767048624522
(PC) Pair completeness is: 0.8660714285714286
(RM) Reference metric (Harmonic mean RR and PC) is: 0.8958627892734415


(RR) Reduction ratio is: 0.9936322475430239
(PC) Pair completeness is: 0.8482142857142857
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9151827276530684
(PQ) Pair quality - Precision is: 0.040016849199663015
(FM) Fmeasure is: 0.07642799678197909
clustering with NUM_CLUSTERS = 181, 
(RR) Reduction ratio is: 0.9938495128964422
(PC) Pair completeness is: 0.8482142857142857
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9152748730252768
(PQ) Pair quality - Precision is: 0.04143044047099869
(FM) Fmeasure is: 0.079002079002079
clustering with NUM_CLUSTERS = 186, 
(RR) Reduction ratio is: 0.9939728981588773
(PC) Pair completeness is: 0.8482142857142857
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9153271928193365
(PQ) Pair quality - Precision is: 0.04227859368046284
(FM) Fmeasure is: 0.08054260279779568
clustering with NUM_CLUSTERS = 191, 
(RR) Reduction ratio is: 0.9941526114759023
(PC) Pair completeness is: 0.8482142857142857
(RM) Reference metric (Harmonic mean RR a

In [10]:
for i in [30,35,40,45,50]:
    print('>>>>>>>>>>>>>>>>>>>>: '+str(i))
    tsne_embeddings = tsne_dim_reduction(
    embeddings_tokens, 
    num_components=2,
    early_exaggeration=12,
    verbose=1,
    perplexity=i,
    method="barnes_hut")
    num_clusters = 56
    blocks = hierarchy_cluster(tsne_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

>>>>>>>>>>>>>>>>>>>>: 30
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 30
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reduction ratio is: 0.9802851808935239
(PC) Pair completeness is: 0.8928571428571429
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9345308305726949
(PQ) Pair quality - Precision is: 0.013605442176870748
(FM) Fmeasure is: 0.02680246582685607
>>>>>>>>>>>>>>>>>>>>: 35
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 35
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reduction ratio is: 0.9804809879404317
(PC) Pair completeness is: 0.8571428571428571
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9146728016466523
(PQ) Pair quality - Precision is: 0.013192249553387384
(FM) Fmeasure is: 0.02598457166057653
>>>>>>>>>>>>>>>>>>>>: 40
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 40
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reducti

In [11]:
for i in [5,10,15]:
    print('>>>>>>>>>>>>>>>>>>>>: '+str(i))
    tsne_embeddings = tsne_dim_reduction(
    embeddings_tokens, 
    num_components=2,
    early_exaggeration=12,
    verbose=1,
    perplexity=i,
    method="barnes_hut")
    num_clusters = 56
    blocks = hierarchy_cluster(tsne_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

>>>>>>>>>>>>>>>>>>>>: 5
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 5
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reduction ratio is: 0.9732951375477448
(PC) Pair completeness is: 0.9464285714285714
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9596738554621366
(PQ) Pair quality - Precision is: 0.01064684612294094
(FM) Fmeasure is: 0.021056813667063964
>>>>>>>>>>>>>>>>>>>>: 10
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 10
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reduction ratio is: 0.9790191408094073
(PC) Pair completeness is: 0.9464285714285714
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9624480383946711
(PQ) Pair quality - Precision is: 0.013551521350038353
(FM) Fmeasure is: 0.026720443660196623
>>>>>>>>>>>>>>>>>>>>: 15
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 15
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reductio

In [12]:
for i in [8,9,11,12]:
    print('>>>>>>>>>>>>>>>>>>>>: '+str(i))
    tsne_embeddings = tsne_dim_reduction(
    embeddings_tokens, 
    num_components=2,
    early_exaggeration=12,
    verbose=1,
    perplexity=i,
    method="barnes_hut")
    num_clusters = 56
    blocks = hierarchy_cluster(tsne_embeddings, {'num_clusters': num_clusters})
    calc_index(blocks,table,pairs)

>>>>>>>>>>>>>>>>>>>>: 8
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 8
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reduction ratio is: 0.9795448693189134
(PC) Pair completeness is: 0.9375
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9580613679769975
(PQ) Pair quality - Precision is: 0.013768686073957514
(FM) Fmeasure is: 0.027138795554406823
>>>>>>>>>>>>>>>>>>>>: 9
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 9
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reduction ratio is: 0.978761641131282
(PC) Pair completeness is: 0.9375
(RM) Reference metric (Harmonic mean RR and PC) is: 0.9576865902496176
(PQ) Pair quality - Precision is: 0.013260924475877747
(FM) Fmeasure is: 0.026151930261519303
>>>>>>>>>>>>>>>>>>>>: 11
starting dimension: 2048
setting TSNE with n_components: 2 & perplexity: 11
early_exaggeration: 12
clustering with NUM_CLUSTERS = 56, 
(RR) Reduction ratio is: 0.980411248444

# Best

In [29]:
#(RR) Reduction ratio is: 0.9923420668640831
#(PC) Pair completeness is: 0.875
#(RM) Reference metric (Harmonic mean RR and PC) is: 0.9299841993751572
#(PQ) Pair quality - Precision is: 0.034325744308231175
#(FM) Fmeasure is: 0.06605999325918437

#['citruname', 'addr']
#{'name': 'Infersent', 'model_type': 'bilstm', 'char_level': False, 'model_version': 2, 'rnn_dim': 1024, 'verbose': 1}
#{'name': 'TSNE', 'num_components': 2, 'early_exaggeration': 12, 'verbose': 1, 'perplexity': 40, 'method': 'barnes_hut'}
#{'name': 'hierarchy', 'num_clusters': 136}